In [0]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
!ls "./drive/My Drive/Dataset/potholedataset"

dryrun_hackathondata.ipynb  test  train


In [1]:
from keras.utils import Sequence
from keras.models import Sequential
from keras.layers import TimeDistributed, Reshape,Permute,MaxPooling2D, LSTM,Dense,Lambda, Activation, BatchNormalization, Bidirectional, GRU, LeakyReLU, Dropout, Conv2D, Flatten, Input
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras import optimizers
import numpy as np 
import cv2 
import os
import random

Using TensorFlow backend.


In [3]:
#if you think you can fit the images in ram, make this a 1...
'''this flag just means whether or not the training images
   will reside in memory or not, otherwise they're read off the
   disk in real time during training..'''

inmemory =1 #just kidding, it's not entirely clear if this is sooo much faster, not worth it . UNLESS ON COLAB, PUT IN MEMORY!!!
if inmemory:
    potholes = [cv2.imread('./drive/My Drive/Dataset/potholedataset/train/positive/resized/%d.jpg' % x,1) for x in range(1661)]
    print ("Original potholes")
    not_potholes = [cv2.imread('./drive/My Drive/Dataset/potholedataset/train/negative/resized/%d.jpg' % x,1) for x in range(1661)] #2364 orig len
    print ("Not potholes done")
    

Original potholes
Not potholes done


In [4]:
'''
    note the sizes of the images are all 299,299, and then the 3 indicates an R, G, and B channel on each image.
'''
if inmemory:
    potholes = np.asarray(potholes)
    not_potholes = np.asarray(not_potholes)
    print (potholes.shape)
    print (not_potholes.shape) 

(1661, 299, 299, 3)
(1661, 299, 299, 3)


In [0]:
#datagenerator compliant with the KERAS api 
trainlen_norm = list(range(1661))
trainlen_not = list(range(1661))
class DataGenTRAIN(Sequence):
        def __init__(self, tot_ex, batch_size = 32):
                self.batch_size = batch_size
                self.tot_ex = tot_ex
                self.on_epoch_end()
                self.lwr = 0
                self.upr = batch_size
                self.half = batch_size
                list_ = list(range(0,self.tot_ex))
                np.random.shuffle(list_)
                self.master = list_
        def __getitem__(self, index):
                indices = self.master[self.lwr:self.upr]
                (X,y) =  self.__data_generation(indices = indices )
                self.lwr += self.half
                self.upr +=self.half
                return (X,y)

        def __len__(self):
                return int(np.floor((self.tot_ex) / self.batch_size))
        def on_epoch_end(self):
                self.lwr = 0
                self.upr = self.batch_size
                return

        def __data_generation(self, indices):
                #X, X_1 are both good and bad images, then they get shuffled along with their indices
                normidxs = random.sample(trainlen_norm, self.batch_size//2) 
                notidxs = random.sample(trainlen_not, self.batch_size//2)
                if inmemory:
                    X= np.array([potholes[x] for x in normidxs])
                else:
                    X= np.array([cv2.imread('./drive/My Drive/Dataset/potholedataset/train/positive/resized/%d.jpg' % x,1) for x in normidxs])
                Y = np.ones(self.batch_size//2)
                
                if inmemory:
                    X_1= np.array([not_potholes[x] for x in notidxs])
                else:
                    X_1= np.array([cv2.imread('./drive/My Drive/Dataset/potholedataset/train/negative/resized/%d.jpg' % x,1) for x in notidxs])
                Y_1 = np.zeros(self.batch_size//2) 
                indices = list(range(self.batch_size))
                np.random.shuffle(indices)
                X = np.append(X,X_1, axis =0)
                Y = np.append(Y,Y_1,axis=0)
                #print(len(X))
                #print(len(Y))
                X = X[indices] 
                Y = Y[indices]                
                return (X,Y)

In [0]:
#we will use InceptionV3 with a custom top and train...
#inception is a stock network that we will top off, these are famous nets that have won the 'ImageNet' competition
inception = InceptionV3(include_top=False, weights='imagenet', classes = 1)
input = Input(shape = (299,299,3))
x = inception(input)
x = Flatten()(x) 
#x = Dense(4, activation = 'relu')(x)
output = Dense(1, activation = 'sigmoid')(x)

In [8]:
model = Model(input = input, output = output)
sgd = optimizers.SGD(lr=0.001, momentum=0.9, decay=1e-6, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd)

traingenerator = DataGenTRAIN(batch_size = 8, tot_ex =1661)

model.fit_generator(generator=traingenerator,workers=1, epochs = 5)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


Epoch 1/5
207/207 [==============================] - 101s 489ms/step - loss: 1.1190
Epoch 2/5
207/207 [==============================] - 87s 420ms/step - loss: 0.3038
Epoch 3/5
207/207 [==============================] - 87s 421ms/step - loss: 0.0735
Epoch 4/5
207/207 [==============================] - 87s 421ms/step - loss: 0.0496
Epoch 5/5
207/207 [==============================] - 87s 422ms/step - loss: 0.0344


In [0]:
# model.fit_generator(generator=traingenerator,workers=1, epochs = 7)

In [10]:
#if you think you can fit the images in ram, make this a 1...
'''this flag just means whether or not the training images
   will reside in memory or not, otherwise they're read off the
   disk in real time during training..'''

testlen_norm = list(range(650))
testlen_not = list(range(1000))

inmemory =1 #just kidding, it's not entirely clear if this is sooo much faster, not worth it . UNLESS ON COLAB, PUT IN MEMORY!!!
if inmemory:
    potholes_test = np.array([cv2.imread('./drive/My Drive/Dataset/potholedataset/test/positive/resized/%d.jpg' % x,1) for x in testlen_norm])
    print ("Original potholes")
    not_potholes_test = np.array([cv2.imread('./drive/My Drive/Dataset/potholedataset/test/negative/resized/%d.jpg' % x,1) for x in testlen_not])
    print ("Not potholes done")
    

Original potholes
Not potholes done


In [11]:
print (potholes_test.shape)
print (not_potholes_test.shape)

(650, 299, 299, 3)
(1000, 299, 299, 3)


In [0]:

gt = np.array([])
class DataGenTEST(Sequence):
        def __init__(self, tot_ex, batch_size = 32):
                self.batch_size = batch_size
                self.tot_ex = tot_ex
                self.lwr = 0
                self.upr = batch_size
                list_ = list(range(0,self.tot_ex))
                np.random.shuffle(list_)
                self.master = list_
        def __getitem__(self, index):
                indices = self.master[self.lwr:self.upr]
                (X,y) =  self.__data_generation(indices = indices )
                self.lwr += self.batch_size
                self.upr +=self.batch_size
                return (X,y)

        def __len__(self):
                return (self.tot_ex) // self.batch_size
        def on_epoch_end(self):
                self.lwr = 0
                self.upr = self.batch_size
                return

        def __data_generation(self, indices):
                normidxs = random.sample(testlen_norm, self.batch_size//2) 
                notidxs = random.sample(testlen_not, self.batch_size//2)
                X= [potholes_test[x] for x in normidxs]
                Y = np.ones(self.batch_size//2)
                X_1= [not_potholes_test[x] for x in notidxs]
                Y_1 = np.zeros(self.batch_size//2) 
                indices = list(range(self.batch_size))
                np.random.shuffle(indices)
                X = np.append(X,X_1, axis =0)
                Y = np.append(Y,Y_1,axis=0)
                #print(len(X))
                #print(len(Y))
                X = X[indices] 
                Y = Y[indices] 
                global gt
                gt = np.append(gt, Y, axis =0)
                return (X,Y)

In [0]:
testgenerator = DataGenTEST(batch_size = 10, tot_ex = 900)

sample = model.predict_generator(generator=testgenerator,workers=1)
for x in range(len(sample)):
    if sample[x] < .5:
        sample[x] =0
    else:
        sample[x] =1

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(gt, sample)

0.7811111111111111